In [1]:
import subprocess
import os
# change
import yaml
import shutil
#
import jaccard.jaccard as jac
import binary_ast.compare_ast as bin_ast

cwd = os.getcwd()


#change
config_file = open('configs/configs.yaml','r')
config = yaml.safe_load(config_file)
processor = config['processor']
baseAddresss = config['baseAddress']
ghidra_head = config['ghidra_head']

p = "ghidra_mods/ghidra_proj/"
p1 = p+"Evolution.rep"
p2 = p+"Evolution.gpr"

if os.path.exists(p1):
    print("Deleting Previous Project Folder/Files for saving new analysis files:")
    shutil.rmtree(p1)
if os.path.exists(p2):
    print("Deleting Previous Project Folder/Files for saving new analysis files:")
    os.remove(p2)
#

os.putenv("GHIDRA_HEADLESS", ghidra_head)

In [3]:
#Import all packages and at base #20004000 and run ghidra analysis to find functions
#Create a new folder for all analyzed binary using the version number
#For all binaries extract generated instructions for all functions and store in a file named info_graph.json under the version folder
# ghidra_folder = Path("./ghidra_mods")

if os.name == 'nt':
    p = subprocess.Popen(["powershell",".\ghidra_mods\ghidra_pipeline.ps1",cwd], stdout=subprocess.PIPE)
    p.communicate()
else:
    subprocess.call(['./ghidra_mods/ghidra_pipeline.sh', cwd, baseAddresss, processor])


openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu122.04, mixed mode)


INFO  Using log config file: jar:file:/home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Framework/Generic/lib/Generic.jar!/generic.log4j.xml (LoggingInitialization)  
INFO  Using log file: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/application.log (LoggingInitialization)  
INFO  Loading user preferences: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/preferences (Preferences)  
INFO  Loading previous preferences: /home/minhajul/.ghidra/.ghidra_10.1-BETA_PUBLIC/preferences (Preferences)  
INFO  Class search complete (2428 ms) (ClassSearcher)  
INFO  Initializing SSL Context (SSLContextInitializer)  
INFO  Initializing Random Number Generator... (SecureRandomFactory)  
INFO  Random Number Generator initialization complete: NativePRNGNonBlocking (SecureRandomFactory)  
INFO  Trust manager disabled, cacerts have not been set (ApplicationTrustManagerFactory)  
INFO  HEADLESS Script Paths:
    /home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Features/VersionTracking/ghidra_scripts
    /home/minhajul/ghid

INFO  -----------------------------------------------------
    ARM Constant Reference Analyzer            2.029 secs
    ARM Pre-Pattern Analyzer                   0.004 secs
    ARM Symbol                                 0.001 secs
    ASCII Strings                              0.030 secs
    Apply Data Archives                        0.116 secs
    Call Convention ID                         0.006 secs
    Call-Fixup Installer                       0.017 secs
    Create Address Tables                      0.014 secs
    Create Function                            0.259 secs
    Data Reference                             0.003 secs
    Decompiler Switch Analysis                 1.548 secs
    Demangler GNU                              0.001 secs
    Disassemble                                0.835 secs
    Disassemble Entry Points                   0.000 secs
    Embedded Media                             0.003 secs
    External Entry References                  0.000 secs
    Function

openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu122.04, mixed mode)


INFO  Using log config file: jar:file:/home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Framework/Generic/lib/Generic.jar!/generic.log4j.xml (LoggingInitialization)  
INFO  Using log file: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/application.log (LoggingInitialization)  
INFO  Loading user preferences: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/preferences (Preferences)  
INFO  Loading previous preferences: /home/minhajul/.ghidra/.ghidra_10.1-BETA_PUBLIC/preferences (Preferences)  
INFO  Class search complete (2784 ms) (ClassSearcher)  
INFO  Initializing SSL Context (SSLContextInitializer)  
INFO  Initializing Random Number Generator... (SecureRandomFactory)  
INFO  Random Number Generator initialization complete: NativePRNGNonBlocking (SecureRandomFactory)  
INFO  Trust manager disabled, cacerts have not been set (ApplicationTrustManagerFactory)  
INFO  HEADLESS Script Paths:
    /home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Features/VersionTracking/ghidra_scripts
    /home/minhajul/ghid

In [4]:
subfolders = [ f.path for f in os.scandir("versions") if f.is_dir() ]
if os.name != 'nt':
    subfolders.reverse()
binary_pairs = [(subfolders[i - 1], subfolders[i]) for i in range(1, len(subfolders))]
print(binary_pairs)

[('versions/otaApp-1_5_4_bin', 'versions/otaApp-1_4_2_bin'), ('versions/otaApp-1_4_2_bin', 'versions/otaApp-1_5_7_bin'), ('versions/otaApp-1_5_7_bin', 'versions/otaApp-1_4_4_bin'), ('versions/otaApp-1_4_4_bin', 'versions/otaApp-1_5_3_bin')]


In [5]:
graph_file_name = "/info_graph.json"

possible_updates_by_binary_pair = {}
for binary_pair in binary_pairs:
    possible_updates = jac.calculate_jaccard(binary_pair[0]+ graph_file_name, binary_pair[1] + graph_file_name, binary_pair[1], 0.85)
    possible_updates_by_binary_pair[binary_pair] = possible_updates
    

In [ ]:
for binary_pair in possible_updates_by_binary_pair:
    binary_a = binary_pair[0].split(os.sep)[1].replace('_', '.')
    binary_b = binary_pair[1].split(os.sep)[1].replace('_', '.')
    func_pairs = possible_updates_by_binary_pair[binary_pair]

    for func_pair in func_pairs:
        print(func_pair)
#         if "0x2000b300" not in func_pair[1]:
#             continue
        
        func_a = func_pair[1][1]
        func_b = func_pair[1][0]
        
        funcA_ast_file = binary_pair[0] + os.sep + func_a + ".json"
        funcB_ast_file = binary_pair[1] + os.sep + func_b + ".json"

        if not os.path.exists(funcA_ast_file):
            if os.name == 'nt':
                p = subprocess.Popen(["powershell",".\ghidra_mods\win_ast.ps1",cwd, binary_a, func_a, binary_pair[0]], stdout=subprocess.PIPE)
                p_o, p_e = p.communicate()
            else:
                subprocess.call(['./ghidra_mods/ast.sh', cwd, binary_a, func_a, binary_pair[0]])
        
        if not os.path.exists(funcB_ast_file):
            if os.name == 'nt':
                p = subprocess.Popen(["powershell",".\ghidra_mods\win_ast.ps1", cwd, binary_b, func_b, binary_pair[1]], stdout=subprocess.PIPE)
                p.communicate()
            else:
                subprocess.call(['./ghidra_mods/ast.sh', cwd, binary_b, func_b, binary_pair[1]])
        print("\n\n\nNew Comparison \n", func_a, func_b)
        path = binary_pair[1] + os.sep + func_a +"_to_" + func_b 
        bin_ast.compare_ast(binary_pair[0], binary_pair[1], funcA_ast_file, funcB_ast_file, path)

([0.0, 0.21416290726817022, 0.857], ['0x1005868', '0x1004d62'])


openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu122.04, mixed mode)


INFO  Using log config file: jar:file:/home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Framework/Generic/lib/Generic.jar!/generic.log4j.xml (LoggingInitialization)  
INFO  Using log file: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/application.log (LoggingInitialization)  
INFO  Loading user preferences: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/preferences (Preferences)  
INFO  Loading previous preferences: /home/minhajul/.ghidra/.ghidra_10.1-BETA_PUBLIC/preferences (Preferences)  
INFO  Class search complete (2526 ms) (ClassSearcher)  
INFO  Initializing SSL Context (SSLContextInitializer)  
INFO  Initializing Random Number Generator... (SecureRandomFactory)  
INFO  Random Number Generator initialization complete: NativePRNGNonBlocking (SecureRandomFactory)  
INFO  Trust manager disabled, cacerts have not been set (ApplicationTrustManagerFactory)  
INFO  HEADLESS Script Paths:
    /home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Features/VersionTracking/ghidra_scripts
    /home/minhajul/ghid

openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu122.04, mixed mode)


INFO  Using log config file: jar:file:/home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Framework/Generic/lib/Generic.jar!/generic.log4j.xml (LoggingInitialization)  
INFO  Using log file: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/application.log (LoggingInitialization)  
INFO  Loading user preferences: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/preferences (Preferences)  
INFO  Loading previous preferences: /home/minhajul/.ghidra/.ghidra_10.1-BETA_PUBLIC/preferences (Preferences)  
INFO  Class search complete (2361 ms) (ClassSearcher)  
INFO  Initializing SSL Context (SSLContextInitializer)  
INFO  Initializing Random Number Generator... (SecureRandomFactory)  
INFO  Random Number Generator initialization complete: NativePRNGNonBlocking (SecureRandomFactory)  
INFO  Trust manager disabled, cacerts have not been set (ApplicationTrustManagerFactory)  
INFO  HEADLESS Script Paths:
    /home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Features/VersionTracking/ghidra_scripts
    /home/minhajul/ghid

...

INFO  Using log config file: jar:file:/home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Framework/Generic/lib/Generic.jar!/generic.log4j.xml (LoggingInitialization)  
INFO  Using log file: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/application.log (LoggingInitialization)  
INFO  Loading user preferences: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/preferences (Preferences)  
INFO  Loading previous preferences: /home/minhajul/.ghidra/.ghidra_10.1-BETA_PUBLIC/preferences (Preferences)  
INFO  Class search complete (2114 ms) (ClassSearcher)  
INFO  Initializing SSL Context (SSLContextInitializer)  
INFO  Initializing Random Number Generator... (SecureRandomFactory)  
INFO  Random Number Generator initialization complete: NativePRNGNonBlocking (SecureRandomFactory)  
INFO  Trust manager disabled, cacerts have not been set (ApplicationTrustManagerFactory)  
INFO  HEADLESS Script Paths:
    /home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Features/VersionTracking/ghidra_scripts
    /home/minhajul/ghid

openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu122.04, mixed mode)


INFO  Using log config file: jar:file:/home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Framework/Generic/lib/Generic.jar!/generic.log4j.xml (LoggingInitialization)  
INFO  Using log file: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/application.log (LoggingInitialization)  
INFO  Loading user preferences: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/preferences (Preferences)  
INFO  Loading previous preferences: /home/minhajul/.ghidra/.ghidra_10.1-BETA_PUBLIC/preferences (Preferences)  
INFO  Class search complete (3560 ms) (ClassSearcher)  
INFO  Initializing SSL Context (SSLContextInitializer)  
INFO  Initializing Random Number Generator... (SecureRandomFactory)  
INFO  Random Number Generator initialization complete: NativePRNGNonBlocking (SecureRandomFactory)  
INFO  Trust manager disabled, cacerts have not been set (ApplicationTrustManagerFactory)  
INFO  HEADLESS Script Paths:
    /home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Features/VersionTracking/ghidra_scripts
    /home/minhajul/ghid

openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu122.04, mixed mode)


INFO  Using log config file: jar:file:/home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Framework/Generic/lib/Generic.jar!/generic.log4j.xml (LoggingInitialization)  
INFO  Using log file: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/application.log (LoggingInitialization)  
INFO  Loading user preferences: /home/minhajul/.ghidra/.ghidra_10.1.5_PUBLIC/preferences (Preferences)  
INFO  Loading previous preferences: /home/minhajul/.ghidra/.ghidra_10.1-BETA_PUBLIC/preferences (Preferences)  
INFO  Class search complete (2744 ms) (ClassSearcher)  
INFO  Initializing SSL Context (SSLContextInitializer)  
INFO  Initializing Random Number Generator... (SecureRandomFactory)  
INFO  Random Number Generator initialization complete: NativePRNGNonBlocking (SecureRandomFactory)  
INFO  Trust manager disabled, cacerts have not been set (ApplicationTrustManagerFactory)  
INFO  HEADLESS Script Paths:
    /home/minhajul/ghidra_10.1.5_PUBLIC/Ghidra/Features/VersionTracking/ghidra_scripts
    /home/minhajul/ghid